## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзей. Одна серия - один документ.

**To do:** 

**1 Создайте индекс этой базы в формате json и в формате матрицы Term-Document**

Компоненты вашей реализации:
    - функция препроцессинга данных
    - функция индексирования данных

**2 С помощью обратного индекса в формате Term-Document посчитайте:** 


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный? 



[download_friends_corpus](https://yadi.sk/d/4wmU7R8JL-k_RA?w=1)

Напоминание про defaultdict: 
> Если ваш код предполагает использование multiple values словаря, рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

> ```from collections import defaultdict
d = defaultdict(list)
d['example'].append('example1')
d['example'].append('example2')```

## Домашнее задание 1

---

### Обход папок

In [1]:
import os
import re

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'friends-data')

In [2]:
os.listdir(filepath)

['.DS_Store',
 'Friends - season 2',
 'Friends - season 5',
 'Friends - season 4',
 'Friends - season 3',
 'Friends - season 6',
 'Friends - season 1',
 'Friends - season 7']

In [3]:
# чтение файлов
data_dict = {}
file_names = []
text_files = []
for root, dirs, files in os.walk(filepath):
    for file in files:
        if file.endswith(".txt"):
            line = ''
            with open(os.path.join(root, file), 'r+', encoding='utf-8') as f:
                content = f.read().splitlines()
            for el in content:
                line += el + ' '
            text_files.append(line)
            names = re.search(r'(Friends.+)\.txt', str(file))
            data_dict[names.group(1)] = line
            file_names.append(names.group(1))

In [4]:
len(data_dict)

165

### Препроцесинг

In [5]:
import string
import pandas as pd
from pymorphy2 import MorphAnalyzer


morph = MorphAnalyzer()

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords

In [8]:
def heros_wrong_lemmatized(line):
    hero = ['рейчело', 'чендлера', 'джоуя']
    hero_correct = ['рейчел', 'чендлер', 'джоуи']
    for name in hero:
        line = line.replace(
            name, hero_correct[hero.index(name)]).replace(
            '- ', ' ').replace(
            '--', ' ').replace(
            '  ', ' ')
    return line

In [9]:
def preprocessing(data):
    data2 = []
    cnt = 0
    for v in data:
        strng = ''
        line = "".join(l for l in v if l not in (
            '.', ',', '?', '!', ':', ';', '—', '--', '<', '>', '"', '(', ')'))
        for word in line.split():
            is_word = re.findall(r'\d+', word)
            eng_word = re.findall(r'[A-Za-z]+', word)
            word = re.sub(r'\ufeff', '', word)
            if len(is_word) == 0 and len(eng_word) == 0:
                wrd = morph.parse(word)[0]
                strng += wrd.normal_form + ' '
        strng = re.sub(r'wwwtvsubtitlesnet', '', strng)
        text = heros_wrong_lemmatized(strng)
        text = ' '.join(
            [wrd for wrd in text.split() if wrd not in stopwords.words(
                "russian")])
        data2.append(text)
    return data2

In [10]:
lemm_text = preprocessing(text_files)

### Получение датафрейма с обычным и лемматизированным текстом

---

делала датафрейм, чтобы можно было визуально сравнить тексты

In [11]:
info = {'file': file_names, 'text': text_files, 'lemmatized_text': lemm_text}

In [12]:
data = pd.DataFrame(info)

### Создание json-а

Оптимальным вариантом мне показалось сделать следюущим образом: для создания словаря, который потом можно будет пололожить в json - пройтись по каждой серии и записать информацию о количестве вхождений каждого слова в отдельный список/колонку даатфрейма, а для матрицы -  сделать корпус из всех текстов сразу.

#### Получение частотного словаря для каждой сериии

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [14]:
def get_word_count(series):
    # составляем корпус документов
    vectorizer = CountVectorizer()
    corpus = [series]
    X = vectorizer.fit_transform(corpus)
    word_list = vectorizer.get_feature_names()
    count_list = X.toarray().sum(axis=0)
    word_dict = dict(zip(word_list, count_list))
    return word_dict

In [15]:
all_data = []
for el in data.lemmatized_text.tolist():
    all_data.append(get_word_count(el))

In [16]:
len(all_data)

165

In [17]:
data['word_count'] = all_data

In [18]:
data

,file,text,lemmatized_text,word_count
0,Friends - 2x08 - The One With The List.ru,"﻿Росс меня поцеловал Боже мой, боже мой, боже...",росс поцеловать бог бог бог верить бог бог бог...,"{'аба': 1, 'ага': 3, 'аж': 1, 'аллергия': 1, '..."
1,Friends - 2x20 - The One Where Old Yeller Dies...,"﻿- Привет, что делаете? - Моника заставляет на...",привет делать моника заставлять смотреть старо...,"{'ага': 1, 'аж': 1, 'алфавит': 1, 'анда': 1, '..."
2,Friends - 2x24 - The One With Barry And Mindy'...,"﻿- Привет, Джоуи! Как прослушивание? - Великол...",привет джоуи прослушивание великолепно встрети...,"{'ага': 1, 'агентша': 1, 'адский': 1, 'актёр':..."
3,Friends - 2x16 - The One Where Joey Moves Out.ru,﻿Идиотизм какой-то У Капитана Хрустика брови ...,идиотизм какой-то капитан хрустика бровь аж шл...,"{'ага': 3, 'аж': 3, 'бегать': 1, 'берк': 1, 'б..."
4,Friends - 2x07 - The One Where Ross Finds Out.ru,"﻿Так,.. что со мной такое? Я выгляжу совсем не...",выглядеть невесело что-то отпугивать пройти ве...,"{'автобус': 1, 'ала': 1, 'алло': 1, 'англичани..."
5,Friends - 2x17 - The One Where Eddie Moves In.ru,"﻿Ну, как вам это? -Каса де Джоуи- - Я сам ту...",это -кас де джоуи весь разукрасить ух джоуи эт...,"{'аквариум': 1, 'альбом': 1, 'балаган': 1, 'бе..."
6,Friends - 2x04 - The One With Phoebe's Husband.ru,"﻿Мам, успокойся, ладно? Это в 10 блоках отсюд...",мама успокоиться ладный это блок отсюда ходить...,"{'абсолютно': 1, 'адно': 1, 'аж': 1, 'академия..."
7,Friends - 2x15 - The One Where Ross And Rachel...,"﻿Так, не подглядывать. Не подглядывать. Не п...",подглядывать подглядывать подглядывать хороший...,"{'австралопитек': 2, 'адам': 1, 'адрес': 1, 'а..."
8,Friends - 2x01 - The One With Ross's New Girlf...,"﻿Ну, вы, наверно, помните, у нас тут много чег...",наверно помнить росс весь ходить влюбить рэйче...,"{'абсолютно': 1, 'автобус': 2, 'ак': 1, 'актри..."
9,Friends - 2x21 - The One With The Bullies.ru,﻿Предприятия и цены на них идут в рост... Мон...,предприятие цена идти рост моника почему смотр...,"{'абсолютно': 1, 'агент': 1, 'акция': 1, 'арти..."


#### Получение списка всех слов в сериале

In [19]:
def all_words(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    word_list = vectorizer.get_feature_names()
    return word_list

In [20]:
all_words_list = all_words(data.lemmatized_text.tolist())

#### Создание словаря для json

In [22]:
json_dict = {}
for word in all_words_list:
    info_list = []
    for dictions in all_data:
        for k, v in dictions.items():
            if word == k:
                info_list.append((file_names[all_data.index(dictions)], v))
        if word not in dictions:
            info_list.append((file_names[all_data.index(dictions)], 0))
    json_dict[str(word)] = info_list

#### Запись в json

In [51]:
import json

with open('all_words.json', 'w', encoding='utf-8-sig') as f:
    json.dump(str(json_dict), f, ensure_ascii=False)

### Создание матрицы

In [25]:
def get_word_count2(corpus):
    # составляем корпус документов
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(
        X.toarray(), columns=vectorizer.get_feature_names())
    return X.toarray(), df

In [26]:
matrix_data = get_word_count2(lemm_text)

In [27]:
matrix = matrix_data[0]

In [28]:
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#### Запись в pkl

In [29]:
import pickle as pkl

In [30]:
with open('matrix.pkl', 'wb') as f:
    pkl.dump(matrix, f)

### Поиск

In [31]:
matrix_df = matrix_data[1]

In [32]:
matrix_df

,аа,ааа,аааа,ааааа,ааааааа,аааааау,аарон,аба,аббатство,абонемент,...,ёвить,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Самое частотное слово

In [33]:
total = matrix_df.sum(axis=0, skipna=True)

In [34]:
max_word = total.sort_values(ascending=False)

In [35]:
print(
    "Most frequent word: '{}' with count {}".format(
        max_word.index[0], max_word.values[0]))

Most frequent word: 'это' with count 7517


#### Самое редкое слово

In [36]:
min_word = max_word.sort_index(ascending=False)

In [37]:
min_word = min_word.sort_values(ascending=True)

Если отсортировать словарь и выбрать слово, которые стоит последним по алфавиту, то получим:

In [38]:
print(
    "Least frequent word: '{}' with count {}".format(
        min_word.index[0], min_word.values[0]))

Least frequent word: 'ёще' with count 1


Но слов, которые встречаются один раз, много, поэтому ниже приведен полный список редких слов:

In [39]:
min_word[min_word.values == 1]

ёще                1
несовершенство     1
несовершенный      1
несобранный        1
несоблазнить       1
несносный          1
несмешной          1
несовместимость    1
нескромный         1
нескладный         1
несексуальный      1
несдержать         1
неряха             1
нерушимый          1
нерожденный        1
нескончаемый       1
неровно            1
несовместимый      1
неспортивный       1
неукоснительно     1
неудачнега         1
неудача            1
нетяпляпистый      1
нетнетнуть         1
несъедобный        1
несом              1
несуществующий     1
нестирать          1
несравненный       1
несравненно        1
                  ..
штив               1
распугать          1
телепрограмма      1
эдный              1
штить              1
таксофон           1
распсиховаться     1
распростронить     1
штолько            1
уоу                1
эдвард             1
щипание            1
эдакий             1
штраф              1
сэвидж             1
растоптать         1
тайком       

In [40]:
# список самых редких слов
list_least_freq = min_word[min_word.values == 1].index.tolist()

In [41]:
len(list_least_freq)

6670

In [42]:
with open('least_freq_words.txt', 'w', encoding='utf-8') as f:
    for el in list_least_freq:
        line = el + '\n'
        f.write(line)

#### Слова во всех документах

In [43]:
matrix_df2 = matrix_df > 0

In [44]:
total2 = matrix_df2.sum(axis=0).sort_values(ascending=False)

In [45]:
total2[total2.values == 165]

мочь      165
ещё       165
просто    165
весь      165
знать     165
это       165
хотеть    165
думать    165
dtype: int64

In [46]:
word_all_texts = total2[total2.values == 165].index.tolist()
wrd_all_txt = ''
for w in word_all_texts:
    wrd_all_txt += w + ' '
print(
    "Word in all texts: {} ".format(
        wrd_all_txt))

Word in all texts: мочь ещё просто весь знать это хотеть думать  


#### Самый популярный герой

In [47]:
heros = total[['моника', 'рэйчел', 'чендлер', 'фиби', 'росс', 'джоуи']]

In [48]:
heros = heros.sort_values(ascending=False)

In [49]:
heros

росс       1016
чендлер     723
моника      684
джоуи       682
фиби        569
рэйчел      237
dtype: int64

In [50]:
print(
    "Most famous character: '{}' with count {}".format(
        heros.index[0], heros.values[0]))

Most famous character: 'росс' with count 1016
